**Cálculadora de diagrama de cortantes y flectores (P9)**
------------------------------------------------------------------------------
***

> Hay que añadir intro. Es el problema del equilibrista, para el cual se pide el cálculo de la expresión analítica de la deflexión. Se usan funciones de calculo simbolico..

In [8]:
from sympy import *             # Librería para trabajo simbólico
import math                     # Librería para utilizar símbolos matemáticos como el número pi, que se escribe como math.pi
import numpy as np              # Librería para poder trabajar con matrices y vectores
import matplotlib.pyplot as plt # Librería para poder dibujar gráficas

x=symbols('x') #variable simbólica coordenada x
R0y,R1y,R2y=symbols('R0y R1y R2y')
M0z,M1z,M2z=symbols('M0z M1z M2z')

A1,B1,A2,B2=symbols('A1 B1 A2 B2')

W=symbols('W')
#W=86



#solicitaciones
P0=0.0 #N  la carga en x=0
P1=0.0 #N  la carga en x=L1
P2=0.0 #N  la carga en x=L1+L2


T0=0.0 #Nm  el par en x=0
T1=0.0 #Nm  la par en x=L1
T2=0.0 #Nm  la par en x=L1+L2+L3


Lt=symbols('Lt')
#Lt=9.1
#cargas distribuidas (positivas en sentido +y)


qq01=-W/Lt #N/m la carga distribuida entre los puntos 0 y 1 (entre x=0 y x=L1)
qq12=-W/Lt #N/m


#geometría
L1=Lt/2 #longitud tramo 01
L2=Lt/2 #longitud tramo 12

#... añadir más si hay más tramos

L=L1+L2 #longitud total

#material

E=symbols('E')
I=symbols('I')

#E=100.0e6
#I=13824*1.0e-8 

#Cálculo estático. Añadir las ecuaciones ΣFy=0 y ΣMz=0 en la primera y segunda linea. Las reacciones que no participen
#en las ecuaciones (R0y,R1y,R2y,M0z,M1z,M2z) hay que añadirlas en las lineas de abajo
#Nota: Todas las reacciones y solicitaciones se consideran positivos en dirección positiva de los ejes cartesianos.
sol,=linsolve([  R1y+integrate(qq01,(x, 0, L)),   #se pone todo positivo siempre
              R2y,
              R0y,
              M0z,
              M1z,
              M2z,
             ], (R0y,R1y,R2y,M0z,M1z,M2z))
print (sol)


#reacciones
R0y=sol[0] #N reacción en y en x=0 
R1y=sol[1] #N reacción en y en x=L1 
R2y=sol[2] #N reacción en y en x=L1+L2 
M0z=sol[3] #Nm reacción momento en x=0 
M1z=sol[4] #Nm reacción momento en x=L1 
M2z=sol[5] #Nm reacción momento en x=L1+L2 


#-------------------------------------------------------------------------------------#
#---------------------------------- CÁLCULO TRAMOS -----------------------------------#
#-------------------------------------------------------------------------------------#

#--------------------------------------TRAMO 1----------------------------------------#
#Definimos la carga distribuida
def q1(x):        
    return  qq01

#Definimos el cortante
VL=R0y+P0 #Este es el valor del cortante a la izquierda del tramo 1, lo que llamamos V+(x=0)
def cortante1(x):        
    return  integrate(q1(x), (x, 0, x)) + VL #esto quiere decir que integramos q(x) dandole a x limites entre 0 y x. 

#Definimos el flector
ML=-M0z-T0 #Este es el valor del flector a la izquierda del tramo 1, lo que llamamos M+(x=0)
def flector1(x):        
    return  integrate(cortante1(x), (x, 0, x)) + ML #en MPa 

V1=cortante1(x) 
M1=flector1(x)
print("La expresión del cortante en el tramo 1 es: ", V1)
print("La expresión del flector en el tramo 1 es: ", M1)


#Elástica
def giro1(x,A1):        
    return  integrate(flector1(x)/(E*I),x) + A1 #en MPa

def deflexion1(x,A1,B1):        
    return  integrate(giro1(x,A1),x) + B1 #en MPa

G1=giro1(x,A1) 
U1=deflexion1(x,A1,B1)
print("La expresión del giro en el tramo 1 es: ", G1)
print("La expresión de la deflexion en el tramo 1 es: ", U1)
print("")


#--------------------------------------TRAMO 2----------------------------------------#
#Definimos la carga distribuida
def q2(x):        
    return  qq12

#Definimos el cortante
VL=V1.subs([(x,L1)]) + R1y + P1 #Este es el valor del cortante a la izquierda del tramo 2, lo que llamamos V+(x=L1) que se 
                          #calcula mediante el equilibrio de la rebanada como el cortante a la izquierda V-(x=L1) más
                          #la carga puntual en x=L1
def cortante2(x):        
    return  integrate(q2(x), (x, L1, x)) + VL #esto quiere decir que integramos q(x) dandole a x limites entre 0 y x. 

#Definimos el flector
ML=M1.subs([(x,L1)]) - M1z - T1 #Este es el valor del flector a la izquierda, lo que llamamos M+(x=L1)
def flector2(x):        
    return  integrate(cortante2(x), (x, L1, x)) + ML #en MPa 

V2=cortante2(x) 
M2=flector2(x)
if L2 !=0:
    print("La expresión del cortante en el tramo 2 es: ", V2)
    print("La expresión del flector en el tramo 2 es: ", M2)

#elástica
def giro2(x):        
    return  integrate(flector2(x)/(E*I),x) + A2 #en MPa

def deflexion2(x):        
    return  integrate(giro2(x),x) + B2 #en MPa

G2=giro2(x) 
U2=deflexion2(x)
if L2 !=0:
    print("La expresión del giro en el tramo 2 es: ", G2)
    print("La expresión de la deflexion en el tramo 2 es: ", U2)
    print("")

    
#-------------------------------------------------------------------------------------#
#----------------------------------- C.C. ELÁSTICA -----------------------------------#
#-------------------------------------------------------------------------------------#    
  
G1_0=G1.subs([(x,0)])    #giro en tramo 1, punto 0
G1_1=G1.subs([(x,L1)])   #giro en tramo 1, punto 1
G2_1=G2.subs([(x,L1)])   #giro en tramo 2, punto 1
G2_2=G2.subs([(x,L1+L2)]) 

U1_0=U1.subs([(x,0)])    #deflexion en tramo 1, punto 0
U1_1=U1.subs([(x,L1)])   #deflexion en tramo 1, punto 1
U2_1=U2.subs([(x,L1)])   #deflexion en tramo 2, punto 1
U2_2=U2.subs([(x,L1+L2)]) 
    
    
sole,=linsolve([  U1_1,      #deflexion en tramo1, punto 1 es nula (apoyo)
                  U2_1,       #deflexion en tramo2, punto 1 es nula (apoyo)
                  G1_1-G2_1, #giro en tramo1 y giro e tramo 2 coincide en el punto 1 (continuidad)
                  G1_1 #giro en tramo1, punto 1 es nulo
             ], (A1,B1,A2,B2))



G1_exp=G1.subs([(A1,sole[0])])
U1_exp=U1.subs([(A1,sole[0]),(B1,sole[1])])
print("Aplicadas las condiciones de contorno...")
print("La expresión del giro en el tramo 1 es: ", G1_exp)
print("La expresión de la deflexion en el tramo 1 es: ", U1_exp)
print("")


G2_exp=G2.subs([(A2,sole[2])])
U2_exp=U2.subs([(A2,sole[2]),(B2,sole[3])])
print("La expresión del giro en el tramo 2 es: ", G2_exp)
print("La expresión de la deflexion en el tramo 2 es: ", U2_exp)
print("")
    
    
print("La deflexión en x=0  es: ", U1_exp.subs([(x,0.0)]))



(0, W, 0, 0, 0, 0)
La expresión del cortante en el tramo 1 es:  -W*x/Lt
La expresión del flector en el tramo 1 es:  -W*x**2/(2*Lt)
La expresión del giro en el tramo 1 es:  A1 - W*x**3/(6*E*I*Lt)
La expresión de la deflexion en el tramo 1 es:  A1*x + B1 - W*x**4/(24*E*I*Lt)

La expresión del cortante en el tramo 2 es:  W - W*x/Lt
La expresión del flector en el tramo 2 es:  -Lt*W/2 + W*x - W*x**2/(2*Lt)
La expresión del giro en el tramo 2 es:  A2 - Lt*W*x/(2*E*I) + W*x**2/(2*E*I) - W*x**3/(6*E*I*Lt)
La expresión de la deflexion en el tramo 2 es:  A2*x + B2 - Lt*W*x**2/(4*E*I) + W*x**3/(6*E*I) - W*x**4/(24*E*I*Lt)

Aplicadas las condiciones de contorno...
La expresión del giro en el tramo 1 es:  Lt**2*W/(48*E*I) - W*x**3/(6*E*I*Lt)
La expresión de la deflexion en el tramo 1 es:  -Lt**3*W/(128*E*I) + Lt**2*W*x/(48*E*I) - W*x**4/(24*E*I*Lt)

La expresión del giro en el tramo 2 es:  7*Lt**2*W/(48*E*I) - Lt*W*x/(2*E*I) + W*x**2/(2*E*I) - W*x**3/(6*E*I*Lt)
La expresión de la deflexion en el tr